In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

/Users/zzjfrank/workspace/programs/anaconda3/envs/croton_web/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
var_df = pd.read_csv("Croton_TTR.csv")
var_df['fs_vep'] = (var_df['Ref. 1bp Frameshift (%)'] + var_df['Ref. 2bp Frameshift (%)'] - 
    var_df['Alt. 1bp Frameshift (%)'] - var_df['Alt. 2bp Frameshift (%)']).abs()

In [3]:
var_df.head()

,Variant ID,Variant Position,Ref. Allele,Alt. Allele,PAM ID,PAM Range,Max Variant Effect (%),Ref. 1bp Insertion (%),Ref. 1bp Frameshift (%),Ref. 2bp Frameshift (%),Alt. 1bp Insertion (%),Alt. 1bp Frameshift (%),Alt. 2bp Frameshift (%),Ref. and Alt. Sequences,fs_vep
0,rs1004021945,31592947,C,A,TTR|17,- : 31592921 - 31592981,12.9,2.1,25.0,52.6,4.8,23.6,40.7,ACATGCACGGCCACATTGATGGCAGGACTGCCTCGGACAGCATCTA...,13.3
1,rs1004021945,31592947,C,A,TTR|18,- : 31592931 - 31592991,4.0,48.1,65.0,18.3,47.1,61.1,18.3,CTTTCTGAACACATGCACGGCCACATTGATGGCAGGACTGCCTCGG...,3.9
2,rs1004021945,31592947,C,A,TTR|16,+ : 31592915 - 31592975,3.4,52.8,63.3,22.9,52.8,61.5,21.3,TCCTCTGATGGTCAAAGTTCTAGATGCTGTCCGAGGCAGTCCTGCC...,3.4
3,rs1004021945,31592947,C,A,TTR|15,- : 31592892 - 31592952,1.7,30.2,51.8,31.7,31.8,52.2,31.2,GCCTCGGACAGCATCTAGAACTTTGACCATCAGAGGACACTTGGAT...,0.1
4,rs1004021945,31592947,C,A,TTR|20,+ : 31592935 - 31592995,1.6,32.9,53.0,29.3,34.5,53.0,29.1,TAGATGCTGTCCGAGGCAGTCCTGCCATCAATGTGGCCGTGCATGT...,0.2


In [4]:
# for reproducing VRL figure
# we should have a better way than this moving forward. FZZ 20230921
exons_lst = []
for _, row in var_df.iterrows():
    varpos = row['Variant Position']
    if (31591903 <= varpos) & (varpos <= 31591971):
        exons_lst.append(1)
    elif (31592896 <= varpos) & (varpos <= 31593026):
        exons_lst.append(2)
    elif (31595120 <= varpos) & (varpos <= 31595255):
        exons_lst.append(3)
    elif (31598568 <= varpos) & (varpos <= 31598821):
        exons_lst.append(4)
    else:
        exons_lst.append(np.nan)
var_df['exon'] = exons_lst

In [5]:
var_df2 = var_df[var_df['exon'].isin((1,2,3,4))]

In [6]:
len(var_df2['PAM ID'].unique())

69

In [7]:
pam_to_stats = {
    key:{
        'max_vep': df['fs_vep'].max(), 
        'fs1': df['Ref. 1bp Frameshift (%)'].mean(),
        'fs2': df['Ref. 2bp Frameshift (%)'].mean(),
        'exon': df['exon'].mean()
    }
    for key, df in var_df2.groupby('PAM ID')
}

In [8]:
pam_to_stats = pd.DataFrame.from_dict(pam_to_stats, orient='index')
pam_to_stats['fs'] = pam_to_stats['fs1'] + pam_to_stats['fs2']

In [9]:
target_pam = 'TTR|24'
pam_to_stats.loc[target_pam]


max_vep     4.9
fs1        45.6
fs2        30.0
exon        2.0
fs         75.6
Name: TTR|24, dtype: float64

In [10]:
print(pam_to_stats.query("exon<=2 and max_vep < 4.9 and fs>75.6").shape[0], "out of", pam_to_stats.query("exon<=2").shape[0])
pam_to_stats.query("exon<=2 and max_vep < 4.9 and fs>75.6")

7 out of 28


,max_vep,fs1,fs2,exon,fs
TTR|1,3.8,56.4,32.2,1.0,88.6
TTR|14,1.7,46.5,33.0,2.0,79.5
TTR|15,3.6,51.8,31.7,2.0,83.5
TTR|20,4.7,53.0,29.3,2.0,82.3
TTR|22,3.7,72.7,16.5,2.0,89.2
TTR|4,1.5,40.7,38.3,1.0,79.0
TTR|5,3.3,78.4,12.6,1.0,91.0


In [11]:
print(pam_to_stats.query("max_vep < 4.9 and fs>75.6").shape[0], "out of",  pam_to_stats.shape[0])
pam_to_stats.query("max_vep < 4.9 and fs>75.6")

20 out of 69


,max_vep,fs1,fs2,exon,fs
TTR|1,3.8,56.4,32.2,1.0,88.6
TTR|14,1.7,46.5,33.0,2.0,79.5
TTR|15,3.6,51.8,31.7,2.0,83.5
TTR|20,4.7,53.0,29.3,2.0,82.3
TTR|22,3.7,72.7,16.5,2.0,89.2
TTR|29,4.2,50.8,30.0,3.0,80.8
TTR|30,2.8,44.4,32.4,3.0,76.8
TTR|33,4.9,32.0,49.1,3.0,81.1
TTR|34,2.9,42.0,35.3,3.0,77.3
TTR|39,2.6,38.9,37.6,3.0,76.5


In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w